# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.233874724435                   -0.50    8.0
  2   -7.250100745686       -1.79       -1.40    1.0
  3   -7.251281408853       -2.93       -2.05    4.0
  4   -7.251241616529   +   -4.40       -2.13    4.0
  5   -7.251321483216       -4.10       -2.55    4.0
  6   -7.251336792303       -4.82       -2.97    3.0
  7   -7.251338786913       -5.70       -4.08    2.0
  8   -7.251338793312       -8.19       -4.20    4.0
  9   -7.251338798544       -8.28       -4.95    3.0
 10   -7.251338798692       -9.83       -5.50    3.0
 11   -7.251338798702      -10.99       -5.92    3.0
 12   -7.251338798704      -11.68       -6.69    2.0
 13   -7.251338798705      -13.28       -7.02    3.0
 14   -7.251338798705      -14.10       -7.50    3.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.06045829005729997
[ Info: Arnoldi iteration step 2: normres = 0.4078910249465892
[ Info: Arnoldi iteration step 3: normres = 0.9671235725647409
[ Info: Arnoldi iteration step 4: normres = 0.274673492792878
[ Info: Arnoldi iteration step 5: normres = 0.5729516695739214
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (4.26e-02, 6.83e-02, 5.24e-01, 2.15e-01, 3.18e-02)
[ Info: Arnoldi iteration step 6: normres = 0.2276614873316351
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (8.33e-03, 1.63e-01, 9.05e-02, 1.14e-01, 1.90e-02)
[ Info: Arnoldi iteration step 7: normres = 0.09412003258559244
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (3.68e-04, 1.25e-02, 6.71e-03, 4.83e-02, 7.68e-02)
[ Info: Arnoldi iteration step 8: normres = 0.08253223198409046
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.31e-05